In [1]:
import pandas as pd
import numpy as np
import os
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, log_loss, hinge_loss
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel, wilcoxon, shapiro
from sklearn.model_selection import GridSearchCV
import gc
import itertools
from sklearn.utils import resample
import ast
import json
import re
from sklearn.dummy import DummyClassifier

import utils 
import model_train
from constants import *


In [2]:
# List of file paths
df = pd.read_csv(path)
df

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,credit_risk
0,<0,6,critical/other existing credit,radio/tv,1169,no known savings,>=7,4,male single,none,...,real estate,67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,existing paid,radio/tv,5951,<100,1<=X<4,2,female div/dep/mar,none,...,real estate,22,none,own,1,skilled,1,none,yes,bad
2,no checking,12,critical/other existing credit,education,2096,<100,4<=X<7,2,male single,none,...,real estate,49,none,own,1,unskilled resident,2,none,yes,good
3,<0,42,existing paid,furniture/equipment,7882,<100,4<=X<7,2,male single,guarantor,...,life insurance,45,none,for free,1,skilled,2,none,yes,good
4,<0,24,delayed previously,new car,4870,<100,1<=X<4,3,male single,none,...,no known property,53,none,for free,2,skilled,2,none,yes,bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,no checking,12,existing paid,furniture/equipment,1736,<100,4<=X<7,3,female div/dep/mar,none,...,real estate,31,none,own,1,unskilled resident,1,none,yes,good
996,<0,30,existing paid,used car,3857,<100,1<=X<4,4,male div/sep,none,...,life insurance,40,none,own,1,high qualif/self emp/mgmt,1,yes,yes,good
997,no checking,12,existing paid,radio/tv,804,<100,>=7,4,male single,none,...,car,38,none,own,1,skilled,1,none,yes,good
998,<0,45,existing paid,radio/tv,1845,<100,1<=X<4,4,male single,none,...,no known property,23,none,for free,1,skilled,1,yes,yes,bad


In [3]:
# Define a list of ML models
models = [
    # ("DT", DecisionTreeClassifier(criterion='entropy', random_state=42)),
    # ("LR", LogisticRegression(solver='lbfgs', max_iter=100, random_state=42, n_jobs=-1)),
    # ("NB", GaussianNB()),
    ("NN", MLPClassifier(hidden_layer_sizes=(20,),solver='adam',batch_size=128,max_iter=50,early_stopping=True,random_state=42)),
    ("RF", RandomForestClassifier(
    n_estimators=4,
    max_depth=5,
    max_features='sqrt',
    criterion='entropy',
    n_jobs=-1,
    warm_start=True,
    random_state=42)),
    # ("SVM", LinearSVC(random_state=42)) 
]

In [ ]:
df_encoded = utils.encode_categorical_from_file(df)

googd_loss = model_train.get_model_standard(df_encoded, n_bootstrap=100, test_size=0.2)
print(f"Good loss: {googd_loss}")

In [4]:
# Initialize an empty DataFrame to store results
results_df = pd.DataFrame()

In [5]:
for name, model in models:
    print(f"Training model: {name}")

    df_encoded = utils.encode_categorical_from_file(df)

    accuracies, precisions, recalls, f1_scores, aucs, losses, tps, tns, fps, fns = model_train.train_model_bootstrap(df_encoded, name, model, n_bootstrap=100, test_size=0.2)

    # Save the results to a dataframe
    result = pd.DataFrame({
        'Accuracy': accuracies,
        'Precision': precisions,
        'Recall': recalls,
        'F1 Score': f1_scores,
        'AUC': aucs,
        'Loss': losses,
        'TP': tps,
        'TN': tns,
        'FP': fps,
        'FN': fns,
        'ML_Model': name
    })

    results_df = pd.concat([results_df, result], ignore_index=False)

Training model: NN
Training model: RF


/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:468: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:468: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:468: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:468: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:468: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:468: UserWarning: Warm-start fitting without increasing n_estim

In [6]:
results_df

,Accuracy,Precision,Recall,F1 Score,AUC,Loss,TP,TN,FP,FN,ML_Model
0,0.675,0.699482,0.950704,0.805970,0.472681,6.949594,135,0,58,7,NN
1,0.695,0.708333,0.964539,0.816817,0.423428,7.653698,136,3,56,5,NN
2,0.640,0.670213,0.926471,0.777778,0.484375,7.900085,126,2,62,10,NN
3,0.660,0.689840,0.928058,0.791411,0.453119,7.203723,129,3,58,10,NN
4,0.660,0.697297,0.914894,0.791411,0.480046,6.712892,129,3,56,12,NN
...,...,...,...,...,...,...,...,...,...,...,...
95,0.775,0.770950,0.971831,0.859813,0.836207,0.473752,138,17,41,4,RF
96,0.755,0.765363,0.951389,0.848297,0.870536,0.456061,137,14,42,7,RF
97,0.770,0.760870,0.985915,0.858896,0.825158,0.484309,140,14,44,2,RF
98,0.720,0.719101,0.955224,0.820513,0.815355,0.515446,128,16,50,6,RF


In [7]:
# Save the results to a CSV file
results_df.to_csv(f"/Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/Baseline experiment results/German_credit_baseline_NN_RF_rerun_results.csv", index=False)